In [1]:
pip install azure-ai-inference

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 5.6 MB/s eta 0:00:00


In [2]:
# pip install azure-ai-inference
import os
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

api_key = "api-key-here"

In [3]:
client = ChatCompletionsClient(
    endpoint='https://DeepSeek-R1-test1.eastus.models.ai.azure.com',
    credential=AzureKeyCredential(api_key)
)

model_info = client.get_model_info()
print("Model name:", model_info.model_name)
print("Model type:", model_info.model_type)
print("Model provider name:", model_info.model_provider_name)

payload = {
  "messages": [
    {
      "role": "user",
      "content": "I am going to Paris, what should I see?"
    },
    {
      "role": "assistant",
      "content": "Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:\n\n1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.\n2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.\n3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.\n\nThese are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world."
    },
    {
      "role": "user",
      "content": "What is so great about #1?"
    }
  ],
  "max_tokens": 2048
}
response = client.complete(payload)

print("Response:", response.choices[0].message.content)
print("Model:", response.model)
print("Usage:")
print("	Prompt tokens:", response.usage.prompt_tokens)
print("	Total tokens:", response.usage.total_tokens)
print("	Completion tokens:", response.usage.completion_tokens)


Model name: DeepSeek-R1
Model type: chat-completion
Model provider name: DeepSeek
Response: <think>
Okay, the user previously asked for recommendations in Paris, and I listed the Eiffel Tower as #1. Now they're asking, "What is so great about #1?" So they want to know why the Eiffel Tower is special.

Hmm, first, I need to recall what makes the Eiffel Tower stand out. It's an iconic symbol of Paris, right? Built for the 1889 World's Fair, initially controversial but now loved. I should mention its history and how it became a symbol. Then, the engineering aspect—Gustave Eiffel's work, the structure's design, how it was the tallest man-made structure for a while. 

Also, the views from the tower are amazing. The different levels you can visit, restaurants like Jules Verne. The fact that it's illuminated at night; maybe talk about the light shows. People get engaged there, take photos, so it's romantic. Cultural impact—movies, art, literature. Events like Bastille Day fireworks. Accessibi

In [8]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

class ChatBot:
    """
    A ChatBot class to interact with an Azure OpenAI model using the ChatCompletionsClient.

    Attributes:
        api_key (str): The API key used to authenticate with the Azure OpenAI API.
        endpoint (str): The endpoint URL for the Azure OpenAI service.
        client (ChatCompletionsClient): An Azure OpenAI client for making requests.
        history (list[dict]): A list of dictionaries representing the conversation history.
    """

    def __init__(self, api_key: str, endpoint: str) -> None:
        """
        Initializes the ChatBot with a given API key and endpoint.
        Also creates a ChatCompletionsClient instance for making requests.

        Args:
            api_key (str): The API key for Azure OpenAI.
            endpoint (str): The endpoint URL for the Azure OpenAI service.
        """
        self.api_key: str = api_key
        self.endpoint: str = endpoint
        self.client: ChatCompletionsClient = ChatCompletionsClient(
            endpoint=self.endpoint,
            credential=AzureKeyCredential(self.api_key)
        )
        self.history: list[dict] = []

    def append_history(self, role: str, content: str) -> None:
        """
        Appends a new message entry to the conversation history.

        Args:
            role (str): The role of the message sender, e.g., "user" or "assistant".
            content (str): The message content to be appended.
        """
        self.history.append({"role": role, "content": content})

    def invoke_api(
        self,
        max_tokens: int = 512,
        temperature: float = 0.7,
        top_p: float = 0.7,
        frequency_penalty: float = 0.0,
        presence_penalty: float = 0.0
    ) -> str:
        """
        Invokes the Azure OpenAI chat API using the stored conversation history.

        Args:
            max_tokens (int, optional): The maximum number of tokens in the response. Defaults to 2048.
            temperature (float, optional): Sampling temperature. Defaults to 0.7.
            top_p (float, optional): Top-p sampling parameter. Defaults to 0.7.
            frequency_penalty (float, optional): Frequency penalty. Defaults to 0.0.
            presence_penalty (float, optional): Presence penalty. Defaults to 0.0.

        Returns:
            str: The assistant's response as a single string.
        """
        payload = {
            "messages": self.history,
            "max_tokens": max_tokens,
            "temperature": temperature,
            "top_p": top_p,
            "frequency_penalty": frequency_penalty,
            "presence_penalty": presence_penalty
        }

        try:
            response = self.client.complete(payload)
            answer: str = self.collapse_response(response)
            return answer
        except Exception as e:
            print(f"❌ An error occurred while invoking the API: {e}")
            return "I'm sorry, but I encountered an error while processing your request."

    def collapse_response(self, response) -> str:
        """
        Extracts the assistant's response from the Azure API response.

        Args:
            response: The response object from the Azure OpenAI API.

        Returns:
            str: The assistant's response content.
        """
        try:
            return response.choices[0].message.content.strip()
        except (AttributeError, IndexError) as e:
            print(f"❌ An error occurred while processing the response: {e}")
            return "I'm sorry, but I couldn't understand the response from the assistant."

    def show_history(self) -> None:
        """
        Prints the entire conversation history with appropriate emojis.
        """
        for entry in self.history:
            role = entry.get("role", "unknown").capitalize()
            content = entry.get("content", "")
            if role.lower() == "user":
                print(f"🧑‍💻 {role}: {content}")
            elif role.lower() == "assistant":
                print(f"🤖 {role}: {content}")
            else:
                print(f"{role}: {content}")


In [9]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

# Assuming the ChatBot class is defined as above or imported from another module
# from chatbot_module import ChatBot

# Replace these with your actual Azure OpenAI credentials
AZURE_API_KEY = api_key
AZURE_ENDPOINT = "https://DeepSeek-R1-test1.eastus.models.ai.azure.com"

# Instantiate the ChatBot
bot = ChatBot(api_key=AZURE_API_KEY, endpoint=AZURE_ENDPOINT)
print("Welcome to the Azure ChatBot! Type 'exit' or 'quit' to end the conversation.\n")

while True:
    try:
        # Get user input
        user_input = input("🧑‍💻 You: ")

        # Check for exit condition
        if user_input.strip().lower() in ["exit", "quit"]:
            print("👋 Ending the conversation. Goodbye!")
            break

        # Append user message to history
        bot.append_history(role="user", content=user_input)

        # Invoke the API to get the assistant's response
        assistant_response = bot.invoke_api()

        # Append assistant response to history
        bot.append_history(role="assistant", content=assistant_response)

        # Display the assistant's response with emoji
        print(f"🤖 Assistant: {assistant_response}\n")

    except KeyboardInterrupt:
        print("\n👋 Conversation interrupted. Goodbye!")
        break
    except Exception as e:
        print(f"❌ An unexpected error occurred: {e}")


Welcome to the Azure ChatBot! Type 'exit' or 'quit' to end the conversation.

🧑‍💻 You: What is one plus one
🤖 Assistant: <think>
Okay, so the user is asking "What is one plus one." Let me start by breaking down the question. They want to know the result of adding the numbers 1 and 1 together.

First, I recall that addition is one of the basic arithmetic operations. When you add two numbers, you're combining their values. In this case, combining 1 and 1. 

Let me visualize it. If I have one apple and someone gives me another apple, how many apples do I have now? That's right, two apples. So 1 apple plus 1 apple equals 2 apples. Translating that back to numbers, 1 + 1 equals 2.

Wait, but maybe the user is looking for a more mathematical explanation. Let me think. In the decimal number system, each digit represents a value, and when you add 1 and 1, the sum is 2 because there's no carrying over involved. The equation is straightforward: 1 + 1 = 2. 

Is there any context where 1 + 1 might